In [10]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [11]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 


In [12]:
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
# Search for tweets within the target date range

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
tweets_list = tweepy.Cursor(api.search_tweets, q="-filter:replies AND -filter:retweets", lang="en", tweet_mode="extended",include_entities=True,count = 100).items(7000)

#return this if you are using pages again!!!!!
'''
for page in ALL_pages:
    tweets_list.extend(page)
'''
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]

'\nfor page in ALL_pages:\n    tweets_list.extend(page)\n'

In [13]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
location = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
lang = []
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
trending_for_area = []

users = [];

print('loading users')
for t in tweets_list:
    tweet.append(t.full_text);
    
    users.append(t.user);
    word_count.append(countWordsInString(t.full_text))
    
    lang.append(t.lang)
    likes.append(t.favorite_count) 
    retweet_count.append(t.retweet_count)
    is_quote_status.append(t.is_quote_status)
    date.append(t.created_at.isoformat()[:10])
    tweet_id.append(t.id)
    hashtags.append(t.entities['hashtags'])
    
i = 0
for x in users:
    if i%25 == 0:
        print("waiting")
        time.sleep(6)
    
    j = x._json
    #opens the page and waits for it to load
    screen_name = j['screen_name']
    country = check_country_real(j['location'])
    
    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count']);
    followers.append(j['followers_count']);
    location.append(country);
    joined.append(x.created_at.isoformat()[:10])
    tweet_count_user.append(x.statuses_count)
    
    trnd = 0
    try:
        trnd = getTrendingTopics(api,country)
    except:
        trnd = []
    
    trending_for_area.append(trnd)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']));
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

10
done


In [14]:
max = len(users)

In [15]:
dick = {"name":user_name[:max],
        "age":age[:max],
        'location':location[:max],
        "gender":gender[:max],
        'join_date':joined[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'date':date[:max],
        'word count': word_count[:max],
        'launguage': lang[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'trending': trending_for_area[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [16]:
df

,name,age,location,gender,join_date,total tweets,followers,following,verified,tweet,likes,retweets,date,word count,launguage,is quote,hashtags,trending,tweet id
0,CUTTYRIC,33.0,FRANCE STREET ☁️9️⃣🇫🇷,None,2009-07-08,88101,1447,565,False,I AINT LIKE HOW NO JUMPER AINT EVEN MENTION 49...,0,0,2023-01-10,40,en,False,"[{'text': 'YALL', 'indices': [121, 126]}]","[Zidane, Le Graet, Zizou, Lloris, Brésil, Desc...",1612698747825168384
1,faridfansjkt48,NaN,None,None,2013-02-15,23272,59,91,False,Faker's found meets the criteria of adequacy–t...,0,0,2023-01-10,12,en,False,[],None,1612698747766509571
2,Applejack_bot,NaN,None,None,2013-06-20,131182,100,0,False,What? Kelp? I don't need kelp. I don't even li...,0,0,2023-01-10,11,en,False,[],None,1612698747506675715
3,DianaAzhar10,NaN,None,None,2013-03-06,6185,59,56,False,"Destiny. Whatever person, smarter. If a person...",0,0,2023-01-10,11,en,False,[],None,1612698747263238150
4,Margare95551357,NaN,None,None,2022-12-16,1366,0,4,False,Do you like my pokemon? 🥺💕 \n#临沂资源 #临沂 https:/...,0,0,2023-01-10,9,en,False,"[{'text': '临沂资源', 'indices': [28, 33]}, {'text...",None,1612698747255029761
5,charles_butter,NaN,None,None,2015-10-28,4835,237,104,False,It has always been $SYS giving back to the com...,0,0,2023-01-10,42,en,True,[],None,1612698747221250054
6,papamarubot,NaN,None,None,2016-01-08,112308,253,116,False,"My, is it alright for me to take lead instead ...",0,0,2023-01-10,14,en,False,[],None,1612698747208871936
7,UBAUganda,NaN,"Kampala, Uganda",None,2016-05-08,7960,8817,157,False,#TuesdayWisdom\n\n“You can take the challenges...,0,0,2023-01-10,28,en,False,"[{'text': 'TuesdayWisdom', 'indices': [0, 14]}]",[],1612698747204505600
8,mo_showtime,NaN,None,None,2014-02-07,121016,693,779,False,you guys post old shit huh lol https://t.co/jp...,0,0,2023-01-10,8,en,True,[],None,1612698747192086529
9,CarmenH95091931,NaN,None,None,2022-12-17,9,0,6,False,Do you like my pokemon? 🥺💕 🌼 134 #北京资源 #北京 ...,0,0,2023-01-10,17,en,False,"[{'text': '北京资源', 'indices': [35, 40]}, {'text...",None,1612698747166937088


In [17]:
DRIVER.quit()

In [18]:
df.to_csv('7k.csv')